In [11]:
import numpy as np
import random as rd
GENERATIONS = 50
POPULATION_SIZE = 100
CHROMOSOME_SIZE = 10
GAP_PROBABILITY = 0.5

class Chromosome():
      def __init__(self):
          sequence = []
          fitness = None
      def setSequence(self, val):
        self.sequence = val
      def setFitness(self, fitness):
        self.fitness = fitness


def fitness(chromosome):
    fitness_ = 0
    for seqOne in range(len(chromosome)):
      for seqTwo in range(seqOne+1, len(chromosome)):
         for i in range(len(chromosome[0])):
           if chromosome[seqOne][i] == chromosome[seqTwo][i] and chromosome[seqOne][i] != '-' and chromosome[seqTwo][i] != '-':
              fitness_+=1 
           if chromosome[seqOne][i] != chromosome[seqTwo][i] and chromosome[seqOne][i] != '-' and chromosome[seqTwo][i] != '-':
              fitness_-=1
            
    return fitness_


def fitness_solo(chromosome, current):
    fitness_ = 0
    for seqOne in range(len(chromosome)):
      if chromosome[seqOne] == current: continue
      for i in range(len(chromosome[0])):
        if chromosome[seqOne][i] == current[i] and chromosome[seqOne][i] != '-' and current[i] != '-':
          fitness_ += 1
        if chromosome[seqOne][i] != current[i] and chromosome[seqOne][i] != '-' and current[i] != '-':
          fitness_ -= 1
      
    return fitness_


def sortGeneration(population):
    population.sort(key=lambda x: x.fitness, reverse=True)
    return population


def generate_chromosome(all_sequence):
    current_index = rd.randint(0, len(all_sequence)-1)
    sequence = all_sequence[current_index]

    size = len(sequence[0])
    chromosome = []
    chromosome.append(sequence[0])
    for i in range(1,len(sequence)):
        currentSequence = sequence[i].copy()
        emptySpaces = size - len(currentSequence)
        indexes = rd.sample(range(size), emptySpaces)
        seq =  [""] * size 
        for j in indexes:
          seq[j] = '-'
        for j in range(len(seq)):
          if seq[j] == '':
            seq[j] = currentSequence.pop(0)
        chromosome.append(seq)
        
    return chromosome


def generatePopulation(all_sequence):
    population = [Chromosome() for i in range(POPULATION_SIZE)]
    for i in range(POPULATION_SIZE):
      population[i].setSequence(generate_chromosome(all_sequence))
      population[i].setFitness(fitness(population[i].sequence))
    return population


def getParents(population, childrens_value):
    parents_arr = []
    i = 0
    while i < childrens_value:
      subset = rd.sample(population,2)
      subset = sortGeneration(subset)
      one = subset[0]
      two = subset[1]
      if one.sequence != two.sequence:
        parents_arr.append([one, two])

        i += 1
    return parents_arr

def get_min_fitness_solo(sequence):
    min = 100
    index = None
    for idx in range(len(sequence)):
      if idx == 0: continue
      fitness_value = fitness_solo(sequence, sequence[idx])
      if fitness_value < min:
        min = fitness_value
        index = idx
    return index

def crossover(parents):
  childrens = []

  for parent in parents:
    one, two = parent
    
    one_idx = get_min_fitness_solo(one.sequence)
    two_idx = get_min_fitness_solo(two.sequence)
      
    temp = one.sequence[one_idx]
    one.sequence[one_idx] = two.sequence[two_idx]
    two.sequence[two_idx] = temp

    new_one = Chromosome()
    new_one.setSequence(one.sequence)
    new_one.setFitness(fitness(new_one.sequence))

    new_two = Chromosome()
    new_two.setSequence(two.sequence)
    new_two.setFitness(fitness(new_two.sequence))
    

    childrens.append(new_one)
    childrens.append(new_two)

  return childrens


def mutation(population):
  for p in population:
    if rd.random() <= GAP_PROBABILITY:
      for index in range(1, len(p.sequence)):
        gap_indexes = [index for index, char in enumerate(p.sequence[index]) if char == '-']
        gap = rd.choice(gap_indexes)
        
        if gap == len(p.sequence[index]) - 1:
          value = len(p.sequence[index]) - 2
        else:
          value = gap+1

        temp = p.sequence[index][gap] 
        p.sequence[index][gap] = p.sequence[index][value]
        p.sequence[index][value] = temp


if __name__ == '__main__':
    all_sequence = []

    all_sequence = [
        [['A', 'A', 'T', 'C', 'G'],
        ['A', 'A', 'T'],
        ['C', 'T'],
         ['A', 'C', 'T']],
        [['T', 'C', 'C', 'G', 'C'],
        ['C', 'G', 'T'],
        ['C', 'A'],
         ['A', 'G', 'C']],
    ]

    population = generatePopulation(all_sequence)

    while GENERATIONS >= 0:
      childrens_value = len(population) // 2
      population = sortGeneration(population)
      
      parents = getParents(population, childrens_value)
      population = crossover(parents)
      mutation(population)
      
      GENERATIONS -= 1


    population.sort(key=lambda x: x.fitness, reverse=True)
    print(population[0].sequence, population[0].fitness)

[['A', 'A', 'T', 'C', 'G'], ['A', 'C', 'T', '-', '-'], ['A', 'A', 'T', '-', '-'], ['A', 'A', 'T', '-', '-']] 12
